In [2]:
# Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
#Load in CSVs from http://nflsavant.com/about.php
data_2020 = "data/pbp-2020.csv"
yr2020_df = pd.read_csv(data_2020)

data_2019 = "data/pbp-2019.csv"
yr2019_df = pd.read_csv(data_2019)

data_2018 = "data/pbp-2018.csv"
yr2018_df = pd.read_csv(data_2018)

data_2017 = "data/pbp-2017.csv"
yr2017_df = pd.read_csv(data_2017)

data_2016 = "data/pbp-2016.csv"
yr2016_df = pd.read_csv(data_2016)

data_2015 = "data/pbp-2015.csv"
yr2015_df = pd.read_csv(data_2015)

data_2014 = "data/pbp-2014.csv"
yr2014_df = pd.read_csv(data_2014)

In [4]:
frames = [yr2020_df, yr2019_df, yr2018_df, yr2017_df, yr2016_df, yr2015_df, yr2014_df]
nfl_data = pd.concat(frames)
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [5]:
nan_value = float("NaN")
nfl_data.replace("", nan_value, inplace=True)
nfl_data.dropna(subset = ["OffenseTeam"], inplace=True)
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [5]:
nfl_data = nfl_data.drop(columns = "Challenger")
#nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [6]:
nfl_data.dropna(how='all', axis=1, inplace=True)

In [7]:
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [9]:
# nfl_data.to_csv("nfldata.csv")

In [15]:
nfl_data.dtypes

GameId                             int64
GameDate                          object
Quarter                            int64
Minute                             int64
Second                             int64
OffenseTeam                       object
DefenseTeam                       object
Down                               int64
ToGo                               int64
YardLine                           int64
SeriesFirstDown                    int64
NextScore                          int64
Description                       object
TeamWin                            int64
SeasonYear                         int64
Yards                              int64
Formation                         object
PlayType                          object
IsRush                             int64
IsPass                             int64
IsIncomplete                       int64
IsTouchdown                        int64
PassType                          object
IsSack                             int64
IsChallenge     

In [8]:
#import nfl scores csv
scores = "data/NFL_Scores.csv"
score_df = pd.read_csv(scores)
score_df

,Score,PtsW,PtsL,PtTot,PtDif,Winner,Losser,Date
0,31-9,31,9,40,22,Tampa Bay Buccaneers,Kansas City Chiefs,2/7/2021
1,31-26,31,26,57,5,Tampa Bay Buccaneers,Green Bay Packers,1/24/2021
2,38-24,38,24,62,14,Kansas City Chiefs,Buffalo Bills,1/24/2021
3,22-17,22,17,39,5,Kansas City Chiefs,Cleveland Browns,1/17/2021
4,30-20,30,20,50,10,Tampa Bay Buccaneers,New Orleans Saints,1/17/2021
...,...,...,...,...,...,...,...,...
677,37-25,37,25,62,12,Detroit Lions,Washington Redskins,10/31/2010
678,59-14,59,14,73,45,Oakland Raiders,Denver Broncos,10/24/2010
679,15-9,15,9,24,6,Pittsburgh Steelers,Atlanta Falcons,9/12/2010
680,51-45,51,45,96,6,Arizona Cardinals,Green Bay Packers,1/10/2010


In [15]:
scores = score_df.rename(columns={"Losser":"Loser"})
scores

,Score,PtsW,PtsL,PtTot,PtDif,Winner,Loser,Date
0,31-9,31,9,40,22,Tampa Bay Buccaneers,Kansas City Chiefs,2/7/2021
1,31-26,31,26,57,5,Tampa Bay Buccaneers,Green Bay Packers,1/24/2021
2,38-24,38,24,62,14,Kansas City Chiefs,Buffalo Bills,1/24/2021
3,22-17,22,17,39,5,Kansas City Chiefs,Cleveland Browns,1/17/2021
4,30-20,30,20,50,10,Tampa Bay Buccaneers,New Orleans Saints,1/17/2021
...,...,...,...,...,...,...,...,...
677,37-25,37,25,62,12,Detroit Lions,Washington Redskins,10/31/2010
678,59-14,59,14,73,45,Oakland Raiders,Denver Broncos,10/24/2010
679,15-9,15,9,24,6,Pittsburgh Steelers,Atlanta Falcons,9/12/2010
680,51-45,51,45,96,6,Arizona Cardinals,Green Bay Packers,1/10/2010


In [12]:
score_df.dtypes

Score     object
PtsW       int64
PtsL       int64
PtTot      int64
PtDif      int64
Winner    object
Losser    object
Date      object
dtype: object

In [13]:
engine = create_engine("sqlite:///nfldata.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
print(Base.classes.keys())
#Save reference to the tables
nfl = Base.classes.nfl
scores = Base.classes.scores

['nfl', 'scores']


In [25]:
#nfl_data.to_sql(name='nfl', con=engine, if_exists='append', index=False)

In [16]:
scores.to_sql(name="scores", con=engine, if_exists='append', index=False)